## Experiments

Pipeline Steps:

*from Driffta*

- Given region and tile
    - load Experiment.json, channelNames.txt, and processingOptions.json
    - for each channel + z
        - construct input image name
        - if not keyence, copy the file first before opening?
        - if channel_arrangement is "color" do max z-projection
        - \* get explanation on what to do with H&E
        - aggregate images into stack
    - choose best focus z-plane for each cycle
        - currently, the same plane is used for all cycles though
    - run drift compensation
    - crop off tile overlap 
        - note that this is different when "useBleachMinimizingCrop" is set in processingOptions
    - run deconvolution
    - set LUTS (for either 3 or 4 channels)
    - do background subtraction if set by experiment.bgSub
    - save result
    - also save single z-stack using best focus plans in separate dir
    


In [1]:
from codex import config

In [ ]:
from skimage import data

# Create same image and offset version of that image
image = data.camera()
shift = (-22.4, 13.32)
offset_image = fourier_shift(np.fft.fftn(image), shift)
offset_image = np.fft.ifftn(offset_image)


def get_tf_config():
    return tf.ConfigProto()


class OpGraph(object):

    def __init__(self, tf_graph, inputs, outputs):
        self.tf_graph = tf_graph
        self.inputs = inputs
        self.outputs = outputs
    
class TensorFlowOp(object):
    
    def __init__(self):
        self.graph = None
        
    def initialize(self):
        self.graph = self._build_graph()
        
    def _build_graph(self):
        raise NotImplementedError()
        
    def _run(self, input_kwargs):
        if self.graph is None:
            raise ValueError('Must initialize operation before running (via `.initialize` method)')

        with tf.Session(config=get_tf_config(), graph=self.graph) as sess:
            args_dict = {self.graph.inputs[k]:v for k, v in input_kwargs.items() if v is not None}
            res = sess.run(self.graph.outputs, feed_dict=args_dict)
            return res
        
class DriftCompensator(object):
    
    def __init__(self, n_dims):
        self.n_dims = n_dims
        
    def _build_graph(self):
        
    
    def run(reference_image, offset_image):
        
        
def tf_phase_correlation(reference_image, offset_image):
    nx, ny = image.shape
    g = tf.Graph()
    with g.as_default():
        # Compute cross-correlation via FFT
        image_product = tf.multiply(tf.fft2d(image), tf.conj(tf.fft2d(offset_image)))
        cc_image = tf.ifft2d(image_product)
        
        # Compute maximum intensity pixel location within 2D cross-correlation array
        # by first flattening and then using argmax 
        cc_arr = tf.reshape(tf.real(cc_image), [-1])
        idx = tf.argmax(cc_arr, axis=0)
        
        # Convert flattened max index to original coordinates
        iy = tf.cast(tf.floor(idx / ny), tf.int64)
        ix = tf.cast((idx % ny) - ny, tf.int64)

        with tf.Session(graph=g) as sess:
            return sess.run([cc_image, iy, ix])

In [2]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=2, threads_per_worker=1,
                       processes=True, memory_limit=16e9)
client = Client(cluster)

try:
    workers = cluster.workers
    client.run(init_logging)
    client.run(set_device, '0', workers=[workers[0].address])
    client.run(set_device, '1', workers=[workers[1].address])
    print('Done set device')
    res = client.map(run_deconvolution, ['/gpu:0', '/gpu:1'])
    #res = client.map(run_mult, ['/cpu:0']*3)
    res = [r.result() for r in res]
    logs = client.get_worker_logs()
finally:
    client.close()
    cluster.close()

Setting device to 0
Setting device to 1


Done set device


KeyboardInterrupt: 

In [40]:
logs[list(logs.keys())[0]]

[['INFO',
  'distributed.worker - INFO -       Start worker at:      tcp://127.0.0.1:52195'],
 ['INFO',
  'distributed.worker - INFO -          Listening to:      tcp://127.0.0.1:52195'],
 ['INFO',
  'distributed.worker - INFO -              nanny at:            127.0.0.1:52188'],
 ['INFO',
  'distributed.worker - INFO - Waiting to connect to:      tcp://127.0.0.1:52186'],
 ['INFO',
  'distributed.worker - INFO - -------------------------------------------------'],
 ['INFO',
  'distributed.worker - INFO -               Threads:                          1'],
 ['INFO',
  'distributed.worker - INFO -                Memory:                   16.00 GB'],
 ['INFO',
  'distributed.worker - INFO -       Local Directory: C:\\Users\\User\\repos\\codex\\python\\notebooks\\experiments\\dask-worker-space\\worker-v3f83e63'],
 ['INFO',
  'distributed.worker - INFO - -------------------------------------------------'],
 ['INFO',
  'distributed.worker - INFO -         Registered to:      tcp://127.0.0.